In [1]:
%load_ext autoreload
%autoreload 2

# Making systems

In [2]:
import molsysmt as msm

In [3]:
from molsysmt import systems
import numpy as np

In [26]:
molsys = msm.convert([systems['nglview']['md_1u19.gro'], systems['nglview']['md_1u19.xtc']],
                      to_form='molsysmt.MolSys')

In [30]:
molsys.topology.groups

,group_id,group_name,group_type,component_index
0,0,ACE,terminal capping,<NA>
1,1,MET,amino acid,<NA>
2,2,ASN,amino acid,<NA>
3,3,GLY,amino acid,<NA>
4,4,THR,amino acid,<NA>
...,...,...,...,...
344,344,GLN,amino acid,<NA>
345,345,VAL,amino acid,<NA>
346,346,ALA,amino acid,<NA>
347,347,PRO,amino acid,<NA>


In [18]:
view = msm.convert(molsys, to_form='nglview.NGLWidget')

In [32]:
msm.convert(view, to_form='molsysmt.Topology')

LRT ['N', 'H', 'CA', 'HA', 'CB', 'HB1', 'HB2', 'CG', 'HG1', 'HG2', 'CD', 'HD1', 'HD2', 'CE', 'HE1', 'HE2', 'NZ', 'HZ', 'C15', 'H15', 'C14', 'H14', 'C20', 'H201', 'H202', 'H203', 'C13', 'C12', 'H12', 'C11', 'H11', 'C10', 'H10', 'C19', 'H191', 'H192', 'H193', 'C9', 'C8', 'H8', 'C7', 'H7', 'C6', 'C18', 'H181', 'H182', 'H183', 'C5', 'C4', 'H41', 'H42', 'C3', 'H31', 'H32', 'C2', 'H21', 'H22', 'C1', 'C16', 'H161', 'H162', 'H163', 'C17', 'H171', 'H172', 'H173', 'C', 'O']


ValueError: 

In [24]:
msm.get(view, element='component', n_components=True)

LRT ['N', 'H', 'CA', 'HA', 'CB', 'HB1', 'HB2', 'CG', 'HG1', 'HG2', 'CD', 'HD1', 'HD2', 'CE', 'HE1', 'HE2', 'NZ', 'HZ', 'C15', 'H15', 'C14', 'H14', 'C20', 'H201', 'H202', 'H203', 'C13', 'C12', 'H12', 'C11', 'H11', 'C10', 'H10', 'C19', 'H191', 'H192', 'H193', 'C9', 'C8', 'H8', 'C7', 'H7', 'C6', 'C18', 'H181', 'H182', 'H183', 'C5', 'C4', 'H41', 'H42', 'C3', 'H31', 'H32', 'C2', 'H21', 'H22', 'C1', 'C16', 'H161', 'H162', 'H163', 'C17', 'H171', 'H172', 'H173', 'C', 'O']


ValueError: 

In [31]:
msm.element.group.small_molecule.get_group_db('LRT')

{'name': '"5-(1,4-diazepan-1-ylsulfonyl)isoquinolin-1-amine"',
 'topology': [{'atoms': ['C',
    'N',
    'O',
    'C1',
    'C10',
    'C11',
    'C12',
    'C13',
    'C2',
    'C3',
    'C4',
    'C5',
    'C6',
    'C7',
    'C8',
    'C9',
    'N1',
    'N2',
    'N3',
    'O1',
    'S',
    'H1',
    'H2',
    'H3',
    'H4',
    'H5',
    'H6',
    'H7',
    'H8',
    'H9',
    'H10',
    'H11',
    'H12',
    'H13',
    'H14',
    'H15',
    'H16',
    'H17',
    'H18'],
   'bonds': [['O', 'S'],
    ['C5', 'C6'],
    ['C5', 'N2'],
    ['C6', 'C7'],
    ['C10', 'C11'],
    ['C10', 'C4'],
    ['S', 'C4'],
    ['S', 'N2'],
    ['S', 'O1'],
    ['C11', 'C12'],
    ['C4', 'C3'],
    ['N2', 'C9'],
    ['C7', 'N3'],
    ['C12', 'C13'],
    ['C3', 'C13'],
    ['C3', 'C2'],
    ['C13', 'C'],
    ['C2', 'C1'],
    ['C', 'N'],
    ['C', 'N1'],
    ['C1', 'N1'],
    ['C9', 'C8'],
    ['N3', 'C8'],
    ['N', 'H1'],
    ['N', 'H2'],
    ['C1', 'H3'],
    ['C10', 'H4'],
    ['C11', 'H5'],
   